In [1]:
# import sys
# !conda install --yes --prefix {sys.prefix} pingouin

In [2]:
import datetime
import numpy as np
import math
import os
import pandas as pd
import pingouin as pg
import re
import seaborn as sns
import json
import statsmodels.api as sm 
import statsmodels.formula.api as smf
import statsmodels.graphics.api as smg
import sys

if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO
    
from IPython.display import HTML
def View(df):
    css = """<style>
    table { border-collapse: collapse; border: 3px solid #eee; }
    table tr th:first-child { background-color: #eeeeee; color: #333; font-weight: bold }
    table thead th { background-color: #eee; color: #000; }
    tr, th, td { border: 1px solid #ccc; border-width: 1px 0 0 1px; border-collapse: collapse;
    padding: 3px; font-family: monospace; font-size: 10px }</style>
    """
    s  = '<script type="text/Javascript">'
    s += 'var win = window.open("", "Title", "toolbar=no, location=no, directories=no, status=no, menubar=no, scrollbars=yes, resizable=yes, width=780, height=200, top="+(screen.height-400)+", left="+(screen.width-840));'
    s += 'win.document.body.innerHTML = \'' + (df.to_html() + css).replace("\n",'\\') + '\';'
    s += '</script>'
    return(HTML(s+css))    

os.chdir(r'C:\Users\User\GitHub\WebET_Analysis')
print("Current Working directory " , os.getcwd())

Current Working directory  C:\Users\User\GitHub\WebET_Analysis


In [3]:
# from IPython.display import HTML

# HTML('''<script>
# code_show=true; 
# function code_toggle() {
#  if (code_show){
#  $('div.input').hide();
#  } else {
#  $('div.input').show();
#  }
#  code_show = !code_show
# } 
# $( document ).ready(code_toggle);
# </script>
# <form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

# Raw Data

## Search for specific subjects

In [81]:
path = 'data_cognition'
subject_files = os.listdir(path)
for i in range(0, len(subject_files)):
    thisSubject = open(path + "/" + subject_files[i]).read()
    if thisSubject.find('5c0e4e3d87e876000151cfec') > (-1):
        print(subject_files[i])

research-study-prolific-126.csv


## Read CSV from String

In [5]:
def cleanhtml(raw_html):
    # https://stackoverflow.com/questions/9662346/python-code-to-remove-html-tags-from-a-string
    cleanr = re.compile('<.*?>')
    cleanText = re.sub(cleanr, '', raw_html)
    return cleanText

def cleanETText(text):
    textWithinBrackets = re.findall(re.compile('\[.*?\]'), text)
    output = text
    for i in range(0,len(textWithinBrackets)):
        old = textWithinBrackets[i]
        new = re.sub(",", "$", old)
        output = output.replace(old, new)
    return output

def cleanSurveyText(text):
    output = text
    textWithinBrackets = re.findall(re.compile('\{.*?\}'), text)
    for i in range(0,len(textWithinBrackets)):
        old = textWithinBrackets[i]
        new = old.replace(',', '§')
        output = output.replace(old, new)
    return output

def compileData(path):
    subject_files = os.listdir(path)
    all_subjects = []
    for i in range(0, len(subject_files)):
        csv_thisSubject = open(path + "/" + subject_files[i]).read()
        csv_thisSubject = cleanhtml(csv_thisSubject)
        csv_thisSubject = cleanETText(csv_thisSubject)
        csv_thisSubject = cleanSurveyText(csv_thisSubject)
        all_subjects.append(pd.read_csv(StringIO(csv_thisSubject)))
    output = pd.concat(all_subjects).reset_index(drop=True)
    return output

#data_yang = compileData("data_yang2020WG")
# Custom modifications
#data_yang.loc[0:515, 'run_id'] = 0
#data_yang = data_yang.loc[~data_yang['run_id'].isin([4, 18]), :]
#data_yang['run_id'] = data_yang['run_id'] * 1000
#data_yang = data_yang.rename(columns={'eyeData': 'et_data'})
#data_cognition = compileData("data_cognition")

#data_raw = data_yang.append(data_cognition)
#data_raw['run_id'].unique()

data_raw = compileData("data_cognition")
# Custom modifications
data_raw['run_id'].unique()

array([1, 103, 105, 106, 108, 11, 117, 12, 121, 124, 125, 126, 128, 13,
       14, 19, 24, 25, 28, 3, 30, 32, 36, 37, 38, 4, 41, 42, 43, 45, 47,
       48, 49, 5, 50, 51, 54, 56, 57, 58, 59, 6, 61, 63, 66, 67, 7, 70,
       72, 74, 75, 80, 81, 83, 85, 88, 9, 90, 91, 92, 93, 94, 96, 97, 98,
       99], dtype=object)

### Exclude empty studies

In [6]:
data_raw = data_raw.loc[data_raw['trial_index']>0, :]

### survey data

In [7]:
def cleanOptionalNote(text):
    optionalNoteText = re.findall(re.compile('optionalNote":.*?\}'), text)
    if len(optionalNoteText) < 1:
        output = text
    else:
        old = optionalNoteText[0]
        new = old.replace('§', ' ')
        output = text.replace(old, new)
    return output


def surveyStringToFrame(subject, string):
    string = cleanOptionalNote(string)
    string = re.sub("""{""", '', string)
    string = re.sub("""}""", '', string)
    string = re.sub('"', '', string)
    string = re.sub('§', '$', string)
    output = pd.read_csv(StringIO(string),
                         sep=":",
                         lineterminator="$",
                         header=None,
                         index_col=0
                         ).transpose()
    return output


def surveyData_thisSubject(data):
    df_thisSubject = data.loc[
        (pd.notna(data["responses"])) &
        (data["responses"] != '"'), :].reset_index()
    subject = df_thisSubject['run_id'].unique()[0]
    output = []
    for i in range(0, len(df_thisSubject)):
        output.append(
            surveyStringToFrame(subject,
                                df_thisSubject.loc[i, 'responses'])
        )

    output = pd.concat(output, axis=1)
    output['run_id'] = subject
    return output

def addSurveyData(data):
    surveyData_allSubjects = pd.DataFrame(columns=[
            'prolificID', 'age', 'gender', 'ethnic', 'sight', 
            'glasses', 'degree', 'eyeshadow', 'masquara', 'eyeliner', 
            'browliner', 'vertPosition', 'triedChin', 'keptHead', 
            'optionalNote', 'run_id'])

    for subject in data['run_id'].unique():
        surveyData_allSubjects = \
            surveyData_allSubjects.append(
                surveyData_thisSubject(
                    data.loc[data['run_id']==subject, ['run_id', 'responses']]
                )
            )
    # print(surveyData_allSubjects)
    
    data = data.merge(surveyData_allSubjects, on='run_id', how='left')
    data = data.drop(columns='responses')
    return(data)
    
data_raw = addSurveyData(data_raw)
print(data_raw['run_id'].unique())
print(data_raw.columns)

[1 103 106 108 11 12 124 125 126 128 13 14 19 24 25 28 30 32 36 37 38 4 41
 42 43 45 47 48 49 5 54 56 58 59 6 61 63 66 67 7 70 72 74 75 80 81 83 85
 88 9 90 91 92 93 94 96 97 98 99]
Index(['run_id', 'condition', 'rt', 'stimulus', 'key_press', 'trial_type',
       'trial_index', 'time_elapsed', 'internal_node_id', 'subject',
       'chinFirst', 'choiceTask_amountLeftFirst', 'webcam_label', 'webcam_fps',
       'webcam_height', 'webcam_width', 'button_pressed', 'window_width',
       'window_height', 'chin', 'success', 'x_pos', 'y_pos', 'task_nr',
       'et_data', 'trial_duration', 'option_topLeft', 'option_bottomLeft',
       'option_topRight', 'option_bottomRight', 'recorded_at', 'ip',
       'user_agent', 'device', 'browser', 'browser_version', 'platform',
       'platform_version', 'Unnamed: 2', 'chosenAmount', 'chosenDelay',
       'prolificID', 'age', 'gender', 'ethnic', 'sight', 'glasses', 'degree',
       'eyeshadow', 'masquara', 'eyeliner', 'browliner', 'vertPosition',
       '

In [8]:
def convertToNumeric(data, columns):
    data[columns] = data[columns].apply(pd.to_numeric, errors='coerce')
    return data


data_raw = convertToNumeric(data_raw, ['age'])
data_raw['degree'] = data_raw['degree'].replace(
        [
            'College / Undergraduate / Bachelor',
            'High School',
            'Graduate / PhD / Master',
            'Middle School'
        ],
    [
            'college',
            'highSchool',
            'grad',
            'middle'
        ]
)
data_raw['run_id'].unique()

array([1, 103, 106, 108, 11, 12, 124, 125, 126, 128, 13, 14, 19, 24, 25,
       28, 30, 32, 36, 37, 38, 4, 41, 42, 43, 45, 47, 48, 49, 5, 54, 56,
       58, 59, 6, 61, 63, 66, 67, 7, 70, 72, 74, 75, 80, 81, 83, 85, 88,
       9, 90, 91, 92, 93, 94, 96, 97, 98, 99], dtype=object)

## Clean raw data

### Failed Webgazer Setup

In [9]:
subjects_failedSetups = []
for subject in data_raw['run_id'].unique():
    maxTrialIndex = data_raw.loc[data_raw['run_id']==subject, ['trial_index']].max()
    if int(maxTrialIndex) < 15:
        subjects_failedSetups.append(subject)

failedSetups = data_raw.loc[
    data_raw['run_id'].isin(subjects_failedSetups), 
    [
        'run_id', 'prolificID', 'chinFirst', 'choiceTask_amountLeftFirst',
        'webcam_label', 'webcam_fps',    
        'user_agent', 'device', 'browser', 'browser_version', 
        'platform', 'platform_version', 
        # 'stimulus', 'trial_type', 'trial_index', 'trial_duration', 
        # 'et_data'
    ]
].drop_duplicates()
    
data_raw = data_raw.loc[~data_raw['run_id'].isin(subjects_failedSetups), :]

print(data_raw['run_id'].unique())
failedSetups

[1 103 11 12 125 126 128 13 19 24 25 28 30 32 36 37 38 4 41 42 43 45 47 48
 5 56 58 59 6 61 63 66 67 7 70 74 75 80 81 85 9 90 91 92 93 94 96 97 98]


,run_id,prolificID,chinFirst,choiceTask_amountLeftFirst,webcam_label,webcam_fps,user_agent,device,browser,browser_version,platform,platform_version
1036,106,5d485e8415055400194b707f,1.0,0.0,"camera2 1, facing front",30.0,Mozilla/5.0 (Linux; Android 10; CLT-L09) Apple...,WebKit,Chrome,87.0.4280.141,AndroidOS,10
1038,108,55b237e6fdf99b19ea79d2f7,0.0,0.0,Front Camera,30.0,Mozilla/5.0 (iPhone; CPU iPhone OS 14_3 like M...,iPhone,Chrome,87.0.4280.77,iOS,14_3
2079,124,56b7a271e77ebe000bbeff49,1.0,1.0,Front Camera,30.0,Mozilla/5.0 (iPhone; CPU iPhone OS 14_3 like M...,iPhone,Safari,14.0.2,iOS,14_3
3653,14,5ec2cdfd1a17930ddf6f1443,1.0,0.0,HD WebCam (04f2:b526),30.0,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,WebKit,Chrome,87.0.4280.141,Windows,10
11953,49,5ecfc227f036c902457fc44c,1.0,0.0,Front Camera,30.0,Mozilla/5.0 (iPhone; CPU iPhone OS 14_2 like M...,iPhone,Safari,14.0.1,iOS,14_2
12476,54,5f94d146f65f56336a61cc16,0.0,0.0,USB2.0 Camera (1e4e:0109),30.0,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,WebKit,Chrome,87.0.4280.141,Windows,10
17307,72,600324757b530e39903effc0,0.0,1.0,"camera2 1, facing front",30.0,Mozilla/5.0 (Linux; Android 9; SM-A105FN) Appl...,WebKit,Chrome,87.0.4280.141,AndroidOS,9
19383,83,600324757b530e39903effc0,0.0,1.0,"camera2 1, facing front",30.0,Mozilla/5.0 (Linux; Android 9; SM-A105FN) Appl...,WebKit,Chrome,87.0.4280.141,AndroidOS,9
19904,88,5ecfc227f036c902457fc44c,0.0,1.0,Front Camera,30.0,Mozilla/5.0 (iPhone; CPU iPhone OS 14_2 like M...,iPhone,Safari,14.0.1,iOS,14_2
24577,99,55b237e6fdf99b19ea79d2f,0.0,0.0,Front Camera,30.0,Mozilla/5.0 (iPhone; CPU iPhone OS 14_3 like M...,iPhone,Safari,14.0.2,iOS,14_3


### No EyeTracking Data

In [10]:
subjects_noet_data = []
for subject in data_raw['run_id'].unique():
    if len(data_raw.loc[data_raw['run_id']==subject, 'et_data'].unique()) < 4:
        subjects_noet_data.append(subject)
print(subjects_noet_data)

data_raw.loc[
    data_raw['run_id'].isin(subjects_noet_data), 
    [
        'run_id', 'prolificID', 'chinFirst', 'choiceTask_amountLeftFirst',
        'webcam_label', 'webcam_fps',
        'user_agent', 'device', 'browser', 'browser_version', 
        'platform', 'platform_version', 
        # 'stimulus', 'trial_type', 'trial_index', 'trial_duration', 
        # 'et_data'
    ]
].drop_duplicates()

[125, 38, 56, 75, 90, 98]


,run_id,prolificID,chinFirst,choiceTask_amountLeftFirst,webcam_label,webcam_fps,user_agent,device,browser,browser_version,platform,platform_version
2083,125,56b7a271e77ebe000bbeff49,1.0,1.0,FaceTime HD Camera (Built-in) (05ac:8600),30.000031,Mozilla/5.0 (Macintosh; Intel Mac OS X 11_1_0)...,Macintosh,Chrome,87.0.4280.88,OS X,11_1_0
7804,38,5f1e9db721e4f101783fe21c,1.0,1.0,OBS Virtual Camera,30.000000,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:8...,NaN,Firefox,84,Windows,10
12480,56,5e79d99f6873570009367716,1.0,0.0,Integrated Webcam (0bda:565a),30.000000,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,WebKit,Chrome,87.0.4280.141,Windows,10
17827,75,5d00bb3b51669b000136a986,0.0,1.0,Lenovo EasyCamera,30.000000,Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:84...,NaN,Firefox,84,Windows,6.1
20426,90,5cf67b30c5cf0e00010c00fc,1.0,1.0,CÃ¡mara FaceTime HD,30.000000,Mozilla/5.0 (Macintosh; Intel Mac OS X 11.1; r...,Macintosh,Firefox,84,OS X,11.1
24058,98,5f779405dacc842fc44bc767,0.0,0.0,NaN,NaN,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:8...,NaN,Firefox,84,Windows,10


### Duplicate Prolific ID's

In [11]:
data_raw.loc[data_raw['run_id']==2, 'prolificID'] = 'Tim2'


duplicates = data_raw.loc[:, ['prolificID', 'trial_index']].duplicated()
duplicateSubjects = data_raw.loc[duplicates, 'run_id'].unique()

if len(duplicateSubjects) > 0:
    print('! Attention: Duplicate subjects: Check out the following: \n')
    print(duplicateSubjects)
else:
    print('Success: No duplicate subjects found')

Success: No duplicate subjects found


## Convert values

In [12]:
# Cleaning
data_raw.loc[: , 
       [
           'option_topLeft',
           'option_bottomLeft', 
           'option_topRight', 
           'option_bottomRight', 
           'chosenAmount', 
           'chosenDelay'
       ] 
    ] = data_raw.loc[:, 
       [
           'option_topLeft',
           'option_bottomLeft', 
           'option_topRight', 
           'option_bottomRight', 
           'chosenAmount', 
           'chosenDelay'
       ] 
    ].replace(['Today', 'Tomorrow', '7 days', '15 days', '30 days', '90 days', '180 days'], 
             [0, 1, 7, 15, 30, 90, 180]) \
    .replace({'\$':''}, regex = True) \
    .replace('50 cent', 0.5) 

In [13]:
data_raw = convertToNumeric(data_raw, 
    [
        'run_id', 'subject', 'chinFirst', 'chin', 'task_nr', 'trial_index', # Int
        'key_press', 
        'x_pos', 'y_pos', 'time_elapsed', 'trial_duration',
        'rt',
        'window_width', 'window_height', 
        'option_topLeft',
        'option_bottomLeft', 
        'option_topRight', 
        'option_bottomRight', 
        'chosenAmount', 
        'chosenDelay'
    ]
)

## Add Variables

### Window Size

In [14]:
def addWindowSize(data):
    output = data
    if "window_width_max" in data.columns:
        print('window width_max already added')
    else:
        grouped = data.groupby(["run_id", "subject"])["window_width", "window_height"].max().reset_index()
        grouped.columns = ["run_id", "subject", "window_width_max", "window_height_max"]
        grouped['window_diagonal_max'] = np.sqrt(grouped['window_width_max']**2 + grouped['window_height_max']**2)
        output = data.merge(grouped, 
                            on=['run_id', "subject"],
                            how='left')
    return output

data_raw = addWindowSize(data_raw)
data_raw['window_diagonal'] = np.sqrt(data_raw['window_width']**2 + 
                                      data_raw['window_height']**2)

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


## Take a look

In [15]:
data_raw.loc[:, ['prolificID', 'run_id']].drop_duplicates().sort_values(by='run_id')

,prolificID,run_id
0,5fccc8ac636416a4288a9f3d,1
8307,5f96fc2552ce3f040af63bbc,4
11937,5ea3f75481a44e39aa318256,5
14011,5f4e08aafb74230a29cb4ef0,6
16357,5fd41bf508717e1833d8aed7,7
19876,5fea6632bf9ae4a79153efdf,9
1036,5c5684ef9d244c0001b29f1e,11
1555,5b8969006651ea000118e42e,12
3124,5b4e65de09f5af0001080f45,13
3643,5f860a189bc695131f22ec9c,19


# data_et

## Compile

In [16]:
def reformatYang(text):
    text = text.replace('$', ',')
    text = text.replace("relative-x", "x")
    text = text.replace("relative-y", 'y')
    text = text.replace('elapse-time', 't')
    text = (text[11:len(text)-1])
    return(text)

for subject in [0, 11000, 21000, 3000, 8000]:
    for i in data_raw.loc[(data_raw['run_id']==subject) &
                      (data_raw['et_data']!='"'), :].index:
        data_raw.loc[i, 'et_data'] = reformatYang(data_raw.loc[i, 'et_data'])

In [32]:
def textToDataframe(text):
    text = text.replace('$', ',')
    dataframe = pd.read_json(text, orient='records')
    return(dataframe)


def extractEyetrackingData(data):
    data_eyetracking = pd.DataFrame(columns=['x', 'y', 't'])
    data["et_data"] = data['et_data'].apply(str)

    for i in data.loc[~(data['et_data'].isin(['"', '[]', 'nan'])), :].index:
        print('Start decoding Index ' + str(i))
        df = textToDataframe(data.loc[i, 'et_data'])        

        df["t_task"] = (df.loc[:, "t"] - df.loc[0, "t"])
        
        df['run_id'] = data.loc[i, 'run_id']
        df['trial_index'] = data.loc[i, 'trial_index']

        data_eyetracking = data_eyetracking.append(pd.DataFrame(data=df), ignore_index=True)
        print('Index ' + str(i) + ' extracted')
    return(data_eyetracking)

data_et = extractEyetrackingData(data_raw)

columnsToAdd = data_raw.columns.drop('et_data')
data_et = data_et.merge(data_raw.loc[:, columnsToAdd], on=['run_id', 'trial_index'], how='left')

data_et.groupby(['run_id', 'chinFirst', 'task_nr',
                 'chin', 'trial_type'])['x'].count()

Start decoding Index 17
Index 17 extracted
Start decoding Index 19
Index 19 extracted
Start decoding Index 21
Index 21 extracted
Start decoding Index 23
Index 23 extracted
Start decoding Index 25
Index 25 extracted
Start decoding Index 27
Index 27 extracted
Start decoding Index 29
Index 29 extracted
Start decoding Index 31
Index 31 extracted
Start decoding Index 33
Index 33 extracted
Start decoding Index 35
Index 35 extracted
Start decoding Index 37
Index 37 extracted
Start decoding Index 39
Index 39 extracted
Start decoding Index 41
Index 41 extracted
Start decoding Index 43
Index 43 extracted
Start decoding Index 45
Index 45 extracted
Start decoding Index 47
Index 47 extracted
Start decoding Index 49
Index 49 extracted
Start decoding Index 51
Index 51 extracted
Start decoding Index 53
Index 53 extracted
Start decoding Index 55
Index 55 extracted
Start decoding Index 57
Index 57 extracted
Start decoding Index 59
Index 59 extracted
Start decoding Index 61
Index 61 extracted
Start decod

Index 334 extracted
Start decoding Index 336
Index 336 extracted
Start decoding Index 337
Index 337 extracted
Start decoding Index 339
Index 339 extracted
Start decoding Index 340
Index 340 extracted
Start decoding Index 342
Index 342 extracted
Start decoding Index 343
Index 343 extracted
Start decoding Index 345
Index 345 extracted
Start decoding Index 346
Index 346 extracted
Start decoding Index 348
Index 348 extracted
Start decoding Index 349
Index 349 extracted
Start decoding Index 351
Index 351 extracted
Start decoding Index 352
Index 352 extracted
Start decoding Index 354
Index 354 extracted
Start decoding Index 355
Index 355 extracted
Start decoding Index 357
Index 357 extracted
Start decoding Index 358
Index 358 extracted
Start decoding Index 360
Index 360 extracted
Start decoding Index 361
Index 361 extracted
Start decoding Index 363
Index 363 extracted
Start decoding Index 364
Index 364 extracted
Start decoding Index 366
Index 366 extracted
Start decoding Index 367
Index 367 

Index 769 extracted
Start decoding Index 771
Index 771 extracted
Start decoding Index 772
Index 772 extracted
Start decoding Index 786
Index 786 extracted
Start decoding Index 787
Index 787 extracted
Start decoding Index 789
Index 789 extracted
Start decoding Index 790
Index 790 extracted
Start decoding Index 792
Index 792 extracted
Start decoding Index 793
Index 793 extracted
Start decoding Index 795
Index 795 extracted
Start decoding Index 796
Index 796 extracted
Start decoding Index 798
Index 798 extracted
Start decoding Index 799
Index 799 extracted
Start decoding Index 801
Index 801 extracted
Start decoding Index 802
Index 802 extracted
Start decoding Index 804
Index 804 extracted
Start decoding Index 805
Index 805 extracted
Start decoding Index 807
Index 807 extracted
Start decoding Index 808
Index 808 extracted
Start decoding Index 810
Index 810 extracted
Start decoding Index 811
Index 811 extracted
Start decoding Index 813
Index 813 extracted
Start decoding Index 814
Index 814 

Index 1127 extracted
Start decoding Index 1129
Index 1129 extracted
Start decoding Index 1140
Index 1140 extracted
Start decoding Index 1141
Index 1141 extracted
Start decoding Index 1143
Index 1143 extracted
Start decoding Index 1144
Index 1144 extracted
Start decoding Index 1146
Index 1146 extracted
Start decoding Index 1147
Index 1147 extracted
Start decoding Index 1149
Index 1149 extracted
Start decoding Index 1150
Index 1150 extracted
Start decoding Index 1152
Index 1152 extracted
Start decoding Index 1153
Index 1153 extracted
Start decoding Index 1155
Index 1155 extracted
Start decoding Index 1156
Index 1156 extracted
Start decoding Index 1158
Index 1158 extracted
Start decoding Index 1159
Index 1159 extracted
Start decoding Index 1161
Index 1161 extracted
Start decoding Index 1162
Index 1162 extracted
Start decoding Index 1164
Index 1164 extracted
Start decoding Index 1165
Index 1165 extracted
Start decoding Index 1178
Index 1178 extracted
Start decoding Index 1180
Index 1180 ex

Index 1462 extracted
Start decoding Index 1464
Index 1464 extracted
Start decoding Index 1465
Index 1465 extracted
Start decoding Index 1467
Index 1467 extracted
Start decoding Index 1468
Index 1468 extracted
Start decoding Index 1470
Index 1470 extracted
Start decoding Index 1471
Index 1471 extracted
Start decoding Index 1473
Index 1473 extracted
Start decoding Index 1474
Index 1474 extracted
Start decoding Index 1476
Index 1476 extracted
Start decoding Index 1477
Index 1477 extracted
Start decoding Index 1479
Index 1479 extracted
Start decoding Index 1480
Index 1480 extracted
Start decoding Index 1482
Index 1482 extracted
Start decoding Index 1483
Index 1483 extracted
Start decoding Index 1485
Index 1485 extracted
Start decoding Index 1486
Index 1486 extracted
Start decoding Index 1488
Index 1488 extracted
Start decoding Index 1489
Index 1489 extracted
Start decoding Index 1491
Index 1491 extracted
Start decoding Index 1492
Index 1492 extracted
Start decoding Index 1494
Index 1494 ex

...

Index 22763 extracted
Start decoding Index 22764
Index 22764 extracted
Start decoding Index 22766
Index 22766 extracted
Start decoding Index 22767
Index 22767 extracted
Start decoding Index 22769
Index 22769 extracted
Start decoding Index 22770
Index 22770 extracted
Start decoding Index 22772
Index 22772 extracted
Start decoding Index 22773
Index 22773 extracted
Start decoding Index 22775
Index 22775 extracted
Start decoding Index 22776
Index 22776 extracted
Start decoding Index 22778
Index 22778 extracted
Start decoding Index 22779
Index 22779 extracted
Start decoding Index 22781
Index 22781 extracted
Start decoding Index 22782
Index 22782 extracted
Start decoding Index 22785
Index 22785 extracted
Start decoding Index 22787
Index 22787 extracted
Start decoding Index 22788
Index 22788 extracted
Start decoding Index 22790
Index 22790 extracted
Start decoding Index 22791
Index 22791 extracted
Start decoding Index 22793
Index 22793 extracted
Start decoding Index 22796
Index 22796 extracte

Index 23152 extracted
Start decoding Index 23153
Index 23153 extracted
Start decoding Index 23155
Index 23155 extracted
Start decoding Index 23156
Index 23156 extracted
Start decoding Index 23158
Index 23158 extracted
Start decoding Index 23159
Index 23159 extracted
Start decoding Index 23161
Index 23161 extracted
Start decoding Index 23162
Index 23162 extracted
Start decoding Index 23164
Index 23164 extracted
Start decoding Index 23165
Index 23165 extracted
Start decoding Index 23167
Index 23167 extracted
Start decoding Index 23168
Index 23168 extracted
Start decoding Index 23170
Index 23170 extracted
Start decoding Index 23171
Index 23171 extracted
Start decoding Index 23173
Index 23173 extracted
Start decoding Index 23174
Index 23174 extracted
Start decoding Index 23176
Index 23176 extracted
Start decoding Index 23177
Index 23177 extracted
Start decoding Index 23179
Index 23179 extracted
Start decoding Index 23180
Index 23180 extracted
Start decoding Index 23182
Index 23182 extracte

Index 23434 extracted
Start decoding Index 23436
Index 23436 extracted
Start decoding Index 23438
Index 23438 extracted
Start decoding Index 23440
Index 23440 extracted
Start decoding Index 23442
Index 23442 extracted
Start decoding Index 23444
Index 23444 extracted
Start decoding Index 23446
Index 23446 extracted
Start decoding Index 23448
Index 23448 extracted
Start decoding Index 23450
Index 23450 extracted
Start decoding Index 23452
Index 23452 extracted
Start decoding Index 23454
Index 23454 extracted
Start decoding Index 23456
Index 23456 extracted
Start decoding Index 23458
Index 23458 extracted
Start decoding Index 23470
Index 23470 extracted
Start decoding Index 23471
Index 23471 extracted
Start decoding Index 23473
Index 23473 extracted
Start decoding Index 23474
Index 23474 extracted
Start decoding Index 23476
Index 23476 extracted
Start decoding Index 23477
Index 23477 extracted
Start decoding Index 23479
Index 23479 extracted
Start decoding Index 23480
Index 23480 extracte

Index 23809 extracted
Start decoding Index 23811
Index 23811 extracted
Start decoding Index 23812
Index 23812 extracted
Start decoding Index 23814
Index 23814 extracted
Start decoding Index 23815
Index 23815 extracted
Start decoding Index 23817
Index 23817 extracted
Start decoding Index 23818
Index 23818 extracted
Start decoding Index 23820
Index 23820 extracted
Start decoding Index 23821
Index 23821 extracted
Start decoding Index 23823
Index 23823 extracted
Start decoding Index 23824
Index 23824 extracted
Start decoding Index 23826
Index 23826 extracted
Start decoding Index 23827
Index 23827 extracted
Start decoding Index 23829
Index 23829 extracted
Start decoding Index 23830
Index 23830 extracted
Start decoding Index 23832
Index 23832 extracted
Start decoding Index 23833
Index 23833 extracted
Start decoding Index 23835
Index 23835 extracted
Start decoding Index 23836
Index 23836 extracted
Start decoding Index 23838
Index 23838 extracted
Start decoding Index 23839
Index 23839 extracte

run_id  chinFirst  task_nr  chin  trial_type             
1.0     1.0        0.0      1.0   eyetracking-calibration     440
                   1.0      1.0   eyetracking-fix-object      162
                   2.0      0.0   eyetracking-calibration     441
                            1.0   eyetracking-choice          628
                                  eyetracking-fix-object      376
                                                             ... 
128.0   0.0        1.0      0.0   eyetracking-fix-object     1237
                            1.0   eyetracking-calibration    4692
                   2.0      1.0   eyetracking-fix-object     1276
                   3.0      1.0   eyetracking-choice         2979
                                  eyetracking-fix-object     2344
Name: x, Length: 255, dtype: int64

In [33]:
data_et = convertToNumeric(data_et,
                        ['x', 'y', 't', 't_task', # Float 
                         'x_pos', 'y_pos', 
                         'chinFirst', 'chin', 'key_press'])

In [34]:
def convertToFactor(data, columns):
    stacked = data[columns].stack()
    data[columns] = pd.Series(stacked.factorize()[0], index=stacked.index).unstack()
    return data

## Add variables

### Count

In [35]:
def addXCount(data):
    if 'count' in data.columns:
        print('Count already added!')
    else: 
        grouped = pd.DataFrame(
                data.groupby(["run_id", "trial_index"])["x"].count()
            ) \
            .reset_index() \
            .rename(columns={'x': 'count'})
        data = data.merge(grouped, 
                          on=["run_id", "trial_index"], 
                          how='left')
    return(data)

data_et = addXCount(data_et)

###  'x', 'y' mean

In [36]:
def addMeans(data):
    if ('x_mean' in data.columns):
        print('X_mean already added!')
        data_output = data
    elif ('y_mean' in data.columns):
        print('Y_mean already added!')
        data_output = data
    else: 
        grouped = data.loc[:, ['run_id', 'subject', 'trial_index', 'x', 'y']] \
                             .groupby(['run_id', 'subject', 'trial_index']).mean() \
                             .rename(columns={'x': 'x_mean', 
                                              'y': 'y_mean'})
        data_output = data.merge(grouped,
                                 on=['run_id', 'subject', 'trial_index'],
                                 how='left')
    return data_output

data_et = addMeans(data_et)

In [37]:
def euclideanDistance(x, x_target, y, y_target):
    x_diff = x - x_target
    y_diff = y - y_target
    euclideanDistance = np.sqrt(x_diff**2 + y_diff**2)
    return(euclideanDistance)

data_et["offset"] = euclideanDistance(data_et["x"], data_et['x_pos'], 
                                      data_et["y"], data_et['y_pos'])

### Precision

In [38]:
def addPrecision(data):
    if 'precision' in data.columns: 
        print('precision already added!')
    else:
        data['deviationFromAVG'] = euclideanDistance(
                data['x'], data['x_mean'], data['y'], data['y_mean']
            )
        data['deviationFromAVG_square'] = np.power(data['deviationFromAVG'], 2)
        grouped = data.groupby(['run_id', 'trial_index']).mean() \
            .reset_index()
        grouped['precision'] = np.sqrt(grouped['deviationFromAVG_square'])
        data = data.merge(
                grouped.loc[:, ['run_id', 'trial_index', 'precision']], 
                on=['run_id', 'trial_index'],
                how='left'
            )
                                
    return data

data_et = addPrecision(data_et)

### WithinTaskIndex

In [39]:
def withinTaskIndex(data): 
    allTrialIndices = []
    for subject in data["run_id"].unique():
        df_subj = data.loc[data['run_id']==subject, :]
        
        for trial_type in df_subj['trial_type'].unique():
            df_trial = df_subj.loc[df_subj['trial_type']==trial_type, :]
                
            for task_nr in df_trial["task_nr"].unique():
                df_thisTask = df_trial.loc[df_trial['task_nr']==task_nr, 
                         [
                             'run_id',
                             'trial_index'
                         ]
                    ] \
                    .drop_duplicates() \
                    .reset_index(drop=True)
        
                df_thisTask['withinTaskIndex'] = df_thisTask.index + 1
                allTrialIndices.append(df_thisTask)
    allTrialIndices = pd.concat(allTrialIndices).reset_index(drop=True)
    return allTrialIndices

def addWithinTaskIndex(data):
    if 'withinTaskIndex' in data.columns: 
        print('withinTaskIndex already added')
    else:
        newIndices = withinTaskIndex(data_et) \
            .reset_index(drop=True)
        data = data.merge(newIndices, 
                          on = ['run_id', 'trial_index'], 
                          how = 'left')
    return data

data_et = addWithinTaskIndex(data_et)

### Position Index

In [40]:
def multiply(x):
    return x*10

def positionIndex(data):
    allPositionIndices = []
    for subject in data["run_id"].unique():
        df_subj = data.loc[
            (
                (data['run_id']==subject) &
                (data['trial_type'].isin(
                        [
                            'eyetracking-calibration', 
                            'eyetracking-fix-object'
                        ]
                    )
                )
            ), :]
        
        for trial_type in df_subj['trial_type'].unique():
            df_trial = df_subj.loc[df_subj['trial_type']==trial_type, :]
                
            for task_nr in df_trial["task_nr"].unique():
                df_thisTask = df_trial.loc[
                        df_trial['task_nr']==task_nr, 
                        ['run_id', 'trial_index', 'x_pos', 'y_pos']
                    ] \
                    .drop_duplicates() \
                    .reset_index(drop=True)

                df_thisTask['positionIndex'] = df_thisTask.loc[:, ['x_pos', 'y_pos']] \
                    .apply(multiply) \
                    .astype(int) \
                    .astype(str) \
                    .apply(''.join, 1)
                df_thisTask['positionIndex'] = df_thisTask['positionIndex'].astype(int)
                df_thisTask['positionIndex'] = df_thisTask.loc[:, 'positionIndex'] \
                    .replace(
                            np.sort(df_thisTask['positionIndex'].unique()), 
                            range(0, len(df_thisTask['positionIndex'].unique()))
                        )        
                allPositionIndices.append(df_thisTask)
                
    allPositionIndices = pd.concat(allPositionIndices).reset_index(drop=True)
    return allPositionIndices

def addPositionIndex(data):
    if 'positionIndex' in data.columns: 
        print('positionIndex already added')
    else:
        newIndices = positionIndex(data_et) \
            .reset_index(drop=True)
        data = data.merge(newIndices, 
                          on = ['run_id', 'trial_index', 'x_pos', 'y_pos'], 
                          how = 'left')
    return data

data_et = addPositionIndex(data_et)

## data_et_calibration

In [42]:
data_et_calibration = data_et.loc[data_et["trial_type"]=="eyetracking-calibration", :]
data_et_calibration

,x,y,t,t_task,run_id,trial_index,condition,rt,stimulus,key_press,...,window_diagonal,count,x_mean,y_mean,offset,deviationFromAVG,deviationFromAVG_square,precision,withinTaskIndex,positionIndex
0,0.260714,0.462839,85956.255,0.000,1.0,18.0,1,NaN,"""",NaN,...,2202.907170,6,0.317473,0.438487,0.599928,0.061763,0.003815,0.068407,1,10.0
1,0.264363,0.461402,86451.430,495.175,1.0,18.0,1,NaN,"""",NaN,...,2202.907170,6,0.317473,0.438487,0.596019,0.057843,0.003346,0.068407,1,10.0
2,0.276912,0.455906,86914.860,958.605,1.0,18.0,1,NaN,"""",NaN,...,2202.907170,6,0.317473,0.438487,0.582330,0.044143,0.001949,0.068407,1,10.0
3,0.307805,0.443429,87395.590,1439.335,1.0,18.0,1,NaN,"""",NaN,...,2202.907170,6,0.317473,0.438487,0.549102,0.010858,0.000118,0.068407,1,10.0
4,0.358234,0.419854,87846.505,1890.250,1.0,18.0,1,NaN,"""",NaN,...,2202.907170,6,0.317473,0.438487,0.493450,0.044818,0.002009,0.068407,1,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501862,0.475341,0.657450,1113590.485,3857.180,97.0,219.0,1,NaN,"""",NaN,...,1509.436981,10,0.407620,0.632198,0.174817,0.072276,0.005224,0.143267,39,9.0
501863,0.385010,0.744614,1114341.385,4608.080,97.0,219.0,1,NaN,"""",NaN,...,1509.436981,10,0.407620,0.632198,0.281375,0.114668,0.013149,0.143267,39,9.0
501864,0.417620,0.775458,1115124.320,5391.015,97.0,219.0,1,NaN,"""",NaN,...,1509.436981,10,0.407620,0.632198,0.264084,0.143609,0.020623,0.143267,39,9.0
501865,0.503810,0.766616,1115957.430,6224.125,97.0,219.0,1,NaN,"""",NaN,...,1509.436981,10,0.407620,0.632198,0.187005,0.165290,0.027321,0.143267,39,9.0


## data_et_fixation

In [43]:
data_et_fixation = data_et.loc[(
           (data_et["trial_type"]=="eyetracking-fix-object") &
           (pd.notna(data_et['withinTaskIndex'])) 
        ), :
    ]
data_et_fixation.columns

Index(['x', 'y', 't', 't_task', 'run_id', 'trial_index', 'condition', 'rt',
       'stimulus', 'key_press', 'trial_type', 'time_elapsed',
       'internal_node_id', 'subject', 'chinFirst',
       'choiceTask_amountLeftFirst', 'webcam_label', 'webcam_fps',
       'webcam_height', 'webcam_width', 'button_pressed', 'window_width',
       'window_height', 'chin', 'success', 'x_pos', 'y_pos', 'task_nr',
       'trial_duration', 'option_topLeft', 'option_bottomLeft',
       'option_topRight', 'option_bottomRight', 'recorded_at', 'ip',
       'user_agent', 'device', 'browser', 'browser_version', 'platform',
       'platform_version', 'Unnamed: 2', 'chosenAmount', 'chosenDelay',
       'prolificID', 'age', 'gender', 'ethnic', 'sight', 'glasses', 'degree',
       'eyeshadow', 'masquara', 'eyeliner', 'browliner', 'vertPosition',
       'triedChin', 'keptHead', 'optionalNote', 'window_width_max',
       'window_height_max', 'window_diagonal_max', 'window_diagonal', 'count',
       'x_mean', 'y_me

## data_et_choice

In [44]:
data_et_choice = data_et.loc[(data_et["trial_type"]=="eyetracking-choice"), :] \
    .reset_index(drop=True)

In [45]:
def lookDirections(data):
    data["look_left"] = (data["x"] < 0.5).astype(int)
    data["look_top"] = (data["y"] < 0.5).astype(int)
    return data

data_et_choice = lookDirections(data_et_choice)

In [46]:
def addChoiceVariables(data):
    data.loc[
                (
                    (data['choiceTask_amountLeftFirst']==1) &
                    (data['withinTaskIndex'] <41)
                ), 'amountLeft'] = 1
    data.loc[
                (
                    (data['choiceTask_amountLeftFirst']==0) &
                    (data['withinTaskIndex'] >40)
                ), 'amountLeft'] = 0

    data.loc[data['amountLeft'] == 1, 'aSS'] = \
        data.loc[data['amountLeft'] == 1,["option_topLeft", "option_bottomLeft"]].values.min(1)
    data.loc[data['amountLeft'] == 0, 'aSS'] = \
        data.loc[data['amountLeft'] == 0,["option_topRight", "option_bottomRight"]].values.min(1)

    data.loc[data['amountLeft'] == 1, 'aLL'] = \
        data.loc[data['amountLeft'] == 1,["option_topLeft", "option_bottomLeft"]].values.max(1)
    data.loc[data['amountLeft'] == 0, 'aLL'] = \
        data.loc[data['amountLeft'] == 0,["option_topRight", "option_bottomRight"]].values.max(1)

    data.loc[:, "tSS"] = 0 

    data.loc[data['amountLeft'] == 1, 'tLL'] = \
        data.loc[data['amountLeft'] == 1,["option_topRight", "option_bottomRight"]].values.max(1)
    data.loc[data['amountLeft'] == 0, 'tLL'] = \
        data.loc[data['amountLeft'] == 0,["option_topLeft", "option_bottomLeft"]].values.max(1)

    data.loc[(data["key_press"]==38), "choseTop"] = 1
    data.loc[(data["key_press"]==40), "choseTop"] = 0

    data.loc[data['amountLeft'] == 1, 'tLL'] = \
        data.loc[data['amountLeft'] == 1,["option_topRight", "option_bottomRight"]].values.max(1)

    data['LL_top'] = (data["option_topLeft"] > data["option_bottomLeft"]).astype(int)

    data.loc[
        (
             (data["choseTop"]==1) & 
             (data["LL_top"] == 1)
        ), "choseLL"] = 1
    data.loc[
        (
             (data["choseTop"]==1) & 
             (data["LL_top"] == 0)
        ), "choseLL"] = 0

    return(data)

data_et_choice = addChoiceVariables(data_et_choice)

In [47]:
def addAOI(data): 
    aoiCenters = pd.DataFrame(
        [
            [(0.05+0.9*0.2), 0.25],
            [(0.05+0.9*0.8), 0.25],
            [(0.05+0.9*0.2), 0.75],
            [(0.05+0.9*0.8), 0.75]
        ], 
        columns = ['x', 'y'],
        index = ['TL', 'TR', 'BL', 'BR']
    )

    for aoi in aoiCenters.index:
        data.loc[
            (
                (data['x'] > (aoiCenters.loc[aoi, 'x'] - 0.175)) & \
                (data['x'] < (aoiCenters.loc[aoi, 'x'] + 0.175)) & \
                (data['y'] > (aoiCenters.loc[aoi, 'y'] - 0.175)) & \
                (data['y'] < (aoiCenters.loc[aoi, 'y'] + 0.175))
             ), 'aoi'] = aoi
    return data 

data_et_choice = addAOI(data_et_choice)

In [48]:
def createAOIColumns(data):
    # If amounts are on the left side
    # If the gaze point is in the top option
    data.loc[((data['amountLeft']==1) & (data['LL_top']==1) & (data['aoi']=='TL')), 
             'aoi_amount_LL'] = 1
    data.loc[((data['amountLeft']==1) & (data['LL_top']==1) & (data['aoi']=='TR')), 
             'aoi_delay_LL'] = 1
    data.loc[((data['amountLeft']==1) & (data['LL_top']==1) & (data['aoi']=='BL')), 
             'aoi_amount_SS'] = 1
    data.loc[((data['amountLeft']==1) & (data['LL_top']==1) & (data['aoi']=='BR')), 
             'aoi_delay_SS'] = 1
    
    data.loc[((data['amountLeft']==1) & (data['LL_top']==0) & (data['aoi']=='TL')), 
             'aoi_amount_SS'] = 1
    data.loc[((data['amountLeft']==1) & (data['LL_top']==0) & (data['aoi']=='TR')), 
             'aoi_delay_SS'] = 1
    data.loc[((data['amountLeft']==1) & (data['LL_top']==0) & (data['aoi']=='BL')), 
             'aoi_amount_LL'] = 1
    data.loc[((data['amountLeft']==1) & (data['LL_top']==0) & (data['aoi']=='BR')), 
             'aoi_delay_LL'] = 1
    
    # If amounts are on the right side
    # If the gaze point is in the top option
    data.loc[((data['amountLeft']==0) & (data['LL_top']==1) & (data['aoi']=='TL')), 
             'aoi_delay_LL'] = 1
    data.loc[((data['amountLeft']==0) & (data['LL_top']==1) & (data['aoi']=='TR')), 
             'aoi_amount_LL'] = 1
    data.loc[((data['amountLeft']==0) & (data['LL_top']==1) & (data['aoi']=='BL')), 
             'aoi_delay_SS'] = 1
    data.loc[((data['amountLeft']==0) & (data['LL_top']==1) & (data['aoi']=='BR')),
             'aoi_amount_SS'] = 1

    data.loc[((data['amountLeft']==0) & (data['LL_top']==0) & (data['aoi']=='TL')), 
             'aoi_delay_SS'] = 1
    data.loc[((data['amountLeft']==0) & (data['LL_top']==0) & (data['aoi']=='TR')), 
             'aoi_amount_SS'] = 1
    data.loc[((data['amountLeft']==0) & (data['LL_top']==0) & (data['aoi']=='BL')), 
             'aoi_delay_LL'] = 1
    data.loc[((data['amountLeft']==0) & (data['LL_top']==0) & (data['aoi']=='BR')), 
             'aoi_amount_LL'] = 1
    return data

data_et_choice = createAOIColumns(data_et_choice)

In [49]:
def add_transition_type(data):
    data = data.loc[(pd.notna(data['aoi'])), :]
    data['newAOIIndex'] = 0
    data.loc[(data['aoi_amount_LL']==1), 'newAOIIndex'] = 1
    data.loc[(data['aoi_delay_LL']==1), 'newAOIIndex'] = 2
    data.loc[(data['aoi_amount_SS']==1), 'newAOIIndex'] = 4
    data.loc[(data['aoi_delay_SS']==1), 'newAOIIndex'] = 8
    data.sort_values(by=['run_id', 'withinTaskIndex'])
    # Add a 0 due to the way np.diff works
    data['transition_type'] = np.append(np.diff(data['newAOIIndex']), [0])
    data['transition_type'] = abs(data['transition_type']) 
    return(data)

def cleanTransitions(data):
    indices = []
    for subject in data['run_id'].unique():
        df_subj = data.loc[data['run_id']==subject, :]
        for trial in df_subj['withinTaskIndex'].unique():
            df_thisTrial = data.loc[(
                                    (data['run_id'] == subject) &
                                    (data['withinTaskIndex'] == trial)
                                ), 'transition_type']
            indices.append(df_thisTrial.index.max())
    # last gaze point of each trial
    data.loc[indices, 'transition_type'] = 0 
    return(data)

data_et_choice = add_transition_type(data_et_choice)

data_et_choice = cleanTransitions(data_et_choice)
# View(data_et_choice.tail(20))

data_et_choice

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\User\Anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

,x,y,t,t_task,run_id,trial_index,condition,rt,stimulus,key_press,...,choseTop,LL_top,choseLL,aoi,aoi_amount_LL,aoi_delay_LL,aoi_amount_SS,aoi_delay_SS,newAOIIndex,transition_type
2,0.622740,0.387974,510486.345,939.735,1.0,145.0,1,4311.830,"""",38.0,...,1.0,1,1.0,TR,NaN,NaN,NaN,NaN,0,0
5,0.396658,0.222469,511919.315,2372.705,1.0,145.0,1,4311.830,"""",38.0,...,1.0,1,1.0,TL,NaN,NaN,NaN,NaN,0,0
49,0.349956,0.349347,552865.440,1985.120,1.0,160.0,1,4445.110,"""",38.0,...,1.0,1,1.0,TL,NaN,NaN,NaN,NaN,0,0
52,0.404848,0.330331,554340.740,3460.420,1.0,160.0,1,4445.110,"""",38.0,...,1.0,1,1.0,TL,NaN,NaN,NaN,NaN,0,0
57,0.595567,0.594855,560653.315,1425.190,1.0,163.0,1,4323.775,"""",38.0,...,1.0,1,1.0,BR,NaN,NaN,NaN,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127315,0.403486,0.309865,2005678.480,2346.895,97.0,493.0,1,3925.945,"""",40.0,...,0.0,0,NaN,TL,NaN,NaN,NaN,1.0,8,0
127316,0.372181,0.276569,2006465.905,3134.320,97.0,493.0,1,3925.945,"""",40.0,...,0.0,0,NaN,TL,NaN,NaN,NaN,1.0,8,0
127329,0.642815,0.241853,2028460.850,1537.260,97.0,502.0,1,3921.230,"""",38.0,...,1.0,1,1.0,TR,1.0,NaN,NaN,NaN,1,0
127330,0.617855,0.249949,2029275.280,2351.690,97.0,502.0,1,3921.230,"""",38.0,...,1.0,1,1.0,TR,1.0,NaN,NaN,NaN,1,0


# data_trial

In [50]:
data_trial = data_raw.loc[
    :, 
    [
        'run_id', 'subject', 'chinFirst', 'choiceTask_amountLeftFirst', 
        'rt', 'stimulus',
        'window_width', 'window_height', 'trial_type', 'trial_index',
        'time_elapsed', 
        'webcam_label', 'webcam_fps',
        'webcam_height', 'webcam_width', 'key_press',
        'success', 'x_pos', 'y_pos', 'chin', 'task_nr',
        'trial_duration', 'option_topLeft', 'option_bottomLeft',
        'option_topRight', 'option_bottomRight', 'recorded_at',
        'user_agent', 'device', 'browser', 'browser_version', 'platform',
        'platform_version', 
        'chosenAmount', 'chosenDelay',
        'prolificID', 'age', 'gender', 'ethnic', 'sight', 'glasses', 'degree',
        'eyeshadow', 'masquara', 'eyeliner', 'browliner', 'vertPosition',
        'triedChin', 'keptHead', 'optionalNote', 'window_width_max',
        'window_height_max', 'window_diagonal_max', 'window_diagonal'
    ]
]

## Add variables from data_et

In [52]:
def mergeByTrialIndex(data, largeData, varName):
    if (varName in data_trial.columns):
        print(varName + ' already added!')
    else:
        grouped = largeData.groupby(['run_id', 'trial_index'])[varName].mean() \
            .reset_index()
        
        
        data = data.merge(grouped, on=['run_id', 'trial_index'], how='left') 
    return data

for column in ['x_mean', 'y_mean', 'count', 'offset', 'precision', 'withinTaskIndex']:
    data_trial = mergeByTrialIndex(data_trial, data_et, column)

### Trial Duration

In [51]:
def checkTimeDeviation(data, column1, column2, maxTimeDiffAllowed):
    diff = data[column1] - data['trial_duration_exact']
    longtrials_runID = data.loc[diff[diff > maxTimeDiffAllowed].index, 'run_id']
    longtrials_previousrunID = pd.DataFrame(data.loc[diff[diff > maxTimeDiffAllowed].index-1, 'run_id']) \
        .rename(columns={'run_id':'previous_run_id'})
    longtrials_previousrunID.index = longtrials_runID.index
    compare_runIDs = pd.concat([longtrials_runID, longtrials_previousrunID], axis=1)

    if sum(compare_runIDs['run_id'] == compare_runIDs['previous_run_id']) > 0: 
        print(column1 + ' and ' + column2 + ' show a deviation of ' +
              '>' + str(maxTimeDiffAllowed) + 
              ' ms. Please check on the following indices: \n')
        print(compare_runIDs.loc[(compare_runIDs['run_id'] == compare_runIDs['previous_run_id']), :].index)

    else:
        print('Success! ' + column1 + ' and ' + column2 + ' do not deviate by ' +
              '>' + str(maxTimeDiffAllowed) + 'ms.')
        
def exactTrialDuration(data):
    output = data
    output["t_startTrial"] = pd.concat([pd.Series([0]), output["time_elapsed"]], ignore_index=True)
    output["trial_duration_exact"] = output.loc[:, ("time_elapsed")] - output.loc[:, ("t_startTrial")]
    output.drop(len(output)-1)
    
    checkTimeDeviation(data, 'rt', 'trial_duration_exact', 50)
    checkTimeDeviation(data, 'trial_duration', 'trial_duration_exact', 50)
    
    return output

data_trial = exactTrialDuration(data_trial)

Success! rt and trial_duration_exact do not deviate by >50ms.
Success! trial_duration and trial_duration_exact do not deviate by >50ms.


### FPS

In [53]:
data_trial['fps'] = 1000 * data_trial['count'] / data_trial['trial_duration_exact']

## data_trial_fixation

In [54]:
data_trial_fixation = data_trial.loc[
    (data_trial['trial_type'] == 'eyetracking-fix-object') &
    (pd.notna(data_et['withinTaskIndex'])) &
    (data_trial['task_nr'] != 3), :] \
    .reset_index(drop=True)
# View(data_trial_fixation.drop(columns=['et_data']).tail(20))

## data_trial_choice

In [55]:
data_trial_choice = data_trial.loc[
    data_trial["trial_type"] == "eyetracking-choice", 
    [
        'run_id', 'chinFirst',
        'trial_duration_exact',
        'window_width', 'window_height', 
        'window_width_max', 'window_height_max',
        'window_diagonal_max', 'window_diagonal', 
        
        'trial_index',  'withinTaskIndex',
        'task_nr',
        'time_elapsed',

        'choiceTask_amountLeftFirst', 
        'key_press', 
        'option_topLeft', 'option_bottomLeft',
        'option_topRight', 'option_bottomRight', 
        'x_mean', 'y_mean', 
        'count', 'fps'        
    ]
]

In [56]:
for column in [
                  'choseTop',
                  'LL_top',
                  'choseLL',
                  'aLL',
                  'aSS',
                  'tLL',
                  'tSS',
                  'amountLeft'
              ]:
    data_trial_choice = mergeByTrialIndex(data_trial_choice, data_et_choice, column)

In [57]:
def addOptionIndex(data):
    if "optionIndex" in data.columns:
        print("Option Index already added!")
        data_output = data
    else:
        grouped = data_et_choice.groupby(['run_id', 'trial_index']) \
            ['aoi', 'aoi_amount_SS', 'aoi_amount_LL', 'aoi_delay_SS', 'aoi_delay_LL'].count() \
            .reset_index() \
            .rename(columns={"aoi": "count"})
        grouped['gazePoints_immediate'] = (grouped['aoi_amount_SS'] + grouped['aoi_delay_SS'])
        grouped['gazePoints_delay'] = (grouped['aoi_amount_LL'] + grouped['aoi_delay_LL'])
        grouped['optionIndex'] = (grouped['gazePoints_immediate'] - grouped['gazePoints_delay']) / \
                                 (grouped['gazePoints_immediate'] + grouped['gazePoints_delay'])
        data_output = data.merge(grouped[['run_id', 'trial_index', 'optionIndex']], 
                                          on=['run_id', 'trial_index'])
    return(data_output)

data_trial_choice = addOptionIndex(data_trial_choice)

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  import sys


In [58]:
def addAttributeIndex(data):
    if "attributeIndex" in data.columns:
        print('Attribute Index already added!')
        data_output = data
    else:
        grouped = data_et_choice.groupby(['run_id', 'trial_index']) \
            ['aoi', 'aoi_amount_SS', 'aoi_amount_LL', 'aoi_delay_SS', 'aoi_delay_LL'].count() \
            .reset_index() \
            .rename(columns={"aoi": "count"})
        grouped['gazePoints_amount'] = (grouped['aoi_amount_LL'] + grouped['aoi_amount_SS'])
        grouped['gazePoints_time'] = (grouped['aoi_delay_LL'] + grouped['aoi_delay_SS'])
        grouped['attributeIndex'] = (grouped['gazePoints_amount'] - grouped['gazePoints_time']) / \
                                 (grouped['gazePoints_amount'] + grouped['gazePoints_time'])

        data_output = data.merge(grouped[['run_id', 'trial_index', 'attributeIndex']], 
                                          on=['run_id', 'trial_index'])
    return(data_output)

data_trial_choice = addAttributeIndex(data_trial_choice)

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  import sys


In [59]:
transition_count = pd.pivot_table(data_et_choice.loc[:, ['run_id', 'trial_index', 'transition_type']], 
                    index = ['run_id', 'trial_index'],
                    columns = ['transition_type'], 
                    aggfunc = len,
                    fill_value = 0
                    ).reset_index()


In [60]:
def addTransition_type(data_trial, data_et):
    if len(data_et.loc[:, 'transition_type'].unique())<2:
        transition_count = pd.DataFrame(
            columns = ['run_id', 'trial_index', 
                    "trans_type_0",
                    "trans_type_1",
                    "trans_type_2",
                    "trans_type_3",
                    "trans_type_4",
                    "trans_type_6",
                    "trans_type_7"]
        )
    else:
        transition_count = pd.pivot_table(data_et.loc[:, ['run_id', 'trial_index', 'transition_type']], 
                            index = ['run_id', 'trial_index'],
                            columns = ['transition_type'], 
                            aggfunc = len,
                            fill_value = 0
                            ).reset_index()
        transition_count.columns = ['run_id', 'trial_index', 
                    "trans_type_0",
                    "trans_type_1",
                    "trans_type_2",
                    "trans_type_3",
                    "trans_type_4",
                    "trans_type_6",
                    "trans_type_7"]

    if "trans_type_0" in data_trial:
        print("Transitions already added!")
        data_trial = data_trial
    else:
        data_trial = data_trial.merge(transition_count, on=['run_id', 'trial_index']) 
    return(data_trial)

data_trial_choice = addTransition_type(data_trial_choice, data_et_choice)

In [61]:
def addPayneIndex(data):
    if "payneIndex" in data.columns:
        print("PayneIndex already added!")
    else: 
        # Option-wise: amount_LL-delay_LL = 1; amount_SS - delay_SS = 4
        # Attribute-wise: amount_LL-amount_SS = 3; delay_LL - delay_SS = 6
        # Cross: amount_LL-delay_SS = 7; delay_LL - amount_SS = 2
        optionWise_transition = data.loc[:, 'trans_type_1'] + data.loc[:, 'trans_type_4']
        attributeWise_transition = data.loc[:, 'trans_type_3'] + data.loc[:, 'trans_type_6']  
        data['payneIndex'] = (optionWise_transition - attributeWise_transition) / \
            (optionWise_transition + attributeWise_transition)      
    return(data)

data_trial_choice = addPayneIndex(data_trial_choice)

In [62]:
def k(aLL, aSS, tLL):
    k = ((aLL / aSS) - 1) / tLL
    return k

data_trial_choice['k'] = k(data_trial_choice['aLL'], data_trial_choice['aSS'], data_trial_choice['tLL']) 

In [63]:
def cleanChoiceData(data):
    output = data.loc[data['trial_duration_exact'] < 10000, :]
    return output

data_trial_choice = cleanChoiceData(data_trial_choice)
# View(data_trial_choice.tail(20)) 

# data_subject

In [64]:
data_subject = data_raw.loc[: , 
          [
               'run_id', 'subject', 'chinFirst',
               'choiceTask_amountLeftFirst', 'webcam_label', 'webcam_fps',
               'webcam_height', 'webcam_width', 
               'user_agent', 'device', 'browser',
               'browser_version', 'platform', 'platform_version', 
               'chosenAmount', 'chosenDelay', 
               'prolificID', 'age',
               'gender', 'ethnic', 'sight', 'glasses', 'degree', 'eyeshadow',
               'masquara', 'eyeliner', 'browliner', 'vertPosition', 'triedChin',
               'keptHead', 'optionalNote', 'window_width_max', 'window_height_max',
               'window_diagonal_max'
          ]
     ].drop_duplicates()

In [65]:
# Add Prolific data
data_prolific = pd.read_csv(r'C:/Users/User/Google Drive/Masterarbeit/Acquise/prolific_documents/prolific_export.csv') \
    .rename(columns=
            {
                'participant_id': 'prolificID',
                'age': 'age_2'
            }
           )

data_subject = data_subject.merge(
    data_prolific, 
    on='prolificID', 
    how='left'
)

In [66]:
if 'date' in data_subject.columns:
    print('date already added!')
else:
    output = []

    for subject in data_trial['prolificID'].unique():
        thisSubject = data_trial.loc[data_trial['prolificID']==subject] \
            .reset_index(drop=True)
        date_time_str=thisSubject.loc[0, 'recorded_at']
        date_time_obj = datetime.datetime.strptime(date_time_str, '%Y-%m-%d %H:%M:%S')

        output.append([thisSubject.loc[0, 'run_id'], subject, date_time_obj.date()])
        
    output = pd.DataFrame(output,
             columns=['run_id', 'prolificID', 'date']
            )   
    data_subject = data_subject.merge(output, on=['run_id', 'prolificID'], how='left')

In [67]:
def merge_group_means_by_subject(data, sourceData, varName):
    if np.invert(varName in sourceData.columns): 
        print(varName + ' not in source Data!')
    else:
        if varName in data.columns:
            print(varName + ' already added!')
        else:
            grouped = sourceData.groupby(['run_id']).mean() \
                .reset_index() \
                .loc[:, ['run_id', varName]]        
            data = data.merge(grouped.loc[:, ['run_id', varName]], on=['run_id'], how='left')
    return data

In [68]:
for column in [
    'trial_duration_exact', 
    't_startTrial', 
    'window_width', 
    'window_height',
    'fps'
               ]:
    data_subject = merge_group_means_by_subject(data_subject, data_trial, column)    

In [69]:
for column in [
    'choseLL'
               ]:
    data_subject = merge_group_means_by_subject(data_subject, data_trial_choice, column)    

In [70]:
for column in [

                   'offset',
                   'precision', 
               ]:
    data_subject = merge_group_means_by_subject(data_subject, data_et, column)    

In [71]:
df = data_et_fixation.rename(columns={
                                                'x_mean': 'x_mean_fixTask',
                                                'y_mean': 'y_mean_fixTask'
                                            })  
for column in ['x_mean_fixTask', 'y_mean_fixTask']:
    data_subject = merge_group_means_by_subject(data_subject, df, column)

In [72]:
columns = [
            'chinFirst',
            'eyeshadow', 
            'masquara',
            'eyeliner',
            'browliner',
            'triedChin', 
            'keptHead',
        ]

data_subject[columns] = data_subject[columns].replace({'no': 0, 'yes': 1}) 
data_subject

,run_id,subject,chinFirst,choiceTask_amountLeftFirst,webcam_label,webcam_fps,webcam_height,webcam_width,user_agent,device,...,trial_duration_exact,t_startTrial,window_width,window_height,fps,choseLL,offset,precision,x_mean_fixTask,y_mean_fixTask
0,1,229.0,1.0,0.0,Microsoft LifeCam VX-3000 (045e:00f5),30.000000,480.0,640.0,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,WebKit,...,3154.351351,8.293924e+05,1920.000000,1078.252708,2.030567,0.625000,0.278014,0.151708,0.537234,0.535832
1,103,153.0,0.0,0.0,Integrated Camera (174f:118d),30.000000,480.0,640.0,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,WebKit,...,71.293436,1.011092e+06,1536.000000,862.462094,18.522002,0.868421,0.140110,0.201333,0.506789,0.442084
2,11,915.0,0.0,1.0,Lenovo EasyCamera (174f:14e8),30.000000,480.0,640.0,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,WebKit,...,-932.314066,6.791403e+05,1280.000000,718.808664,21.456080,0.304348,0.219583,0.180708,0.539989,0.356839
3,12,908.0,0.0,1.0,Fotocamera HD FaceTime,30.000000,480.0,640.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6...,Macintosh,...,341.380309,7.423814e+05,1280.000000,798.407942,17.997210,0.972973,0.279895,0.009254,0.352149,0.374250
4,125,36.0,1.0,1.0,FaceTime HD Camera (Built-in) (05ac:8600),30.000031,480.0,640.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 11_1_0)...,Macintosh,...,-76849.400000,1.805462e+05,1440.000000,726.000000,NaN,NaN,NaN,NaN,NaN,NaN
5,126,305.0,0.0,1.0,Integrated Webcam (1bcf:2895),30.000000,480.0,640.0,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,WebKit,...,2020.405405,7.407898e+05,1600.000000,898.906137,11.808102,0.972222,0.120721,0.146419,0.521822,0.559653
6,128,301.0,0.0,1.0,XiaoMi USB 2.0 Webcam (04f2:b5a3),30.000000,480.0,640.0,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,WebKit,...,613.194981,9.561610e+05,1280.000000,718.462094,21.062559,0.951220,0.164023,0.161438,0.515801,0.532353
7,13,520.0,1.0,0.0,USB2.0 HD UVC WebCam (13d3:56a2),30.000000,480.0,640.0,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,WebKit,...,-617.233141,6.460131e+05,2132.512635,1198.299639,21.704953,0.777778,0.170985,0.174057,0.605638,0.517575
8,19,603.0,0.0,0.0,USB 2.0 Webcam Device (04f2:b521),30.000000,480.0,640.0,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,WebKit,...,189.930636,7.347940e+05,1920.000000,1078.797834,21.565793,0.833333,0.151510,0.151707,0.521838,0.503481
9,24,338.0,1.0,0.0,Full HD webcam (1bcf:2284),30.000000,480.0,640.0,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,WebKit,...,1006.212355,1.113784e+06,1279.927798,720.873646,15.268646,0.567568,0.174037,0.166180,0.541618,0.423060


In [73]:
# View(data_subject)

# Add data_subject variables to data_trial variables

In [74]:
def merge_uniques_by_subject(data, sourceData, varName):
    if np.invert(varName in sourceData.columns): 
        print(varName + ' not in source Data!')
    else:
        if varName in data.columns:
            print(varName + ' already added!')
        else:
            grouped = sourceData.loc[:, 
                [
                    'run_id', varName
                ]
            ] \
            .reset_index(drop=True) \
            .drop_duplicates()     
            data = data.merge(grouped.loc[:, ['run_id', varName]], on=['run_id'], how='left')
    return data

for column in ['age', 'gender', 'ethnic', 'degree']:
    data_trial_choice = merge_uniques_by_subject(data_trial_choice, data_subject, column)
data_trial_choice

,run_id,chinFirst,trial_duration_exact,window_width,window_height,window_width_max,window_height_max,window_diagonal_max,window_diagonal,trial_index,...,trans_type_3,trans_type_4,trans_type_6,trans_type_7,payneIndex,k,age,gender,ethnic,degree
0,1,1.0,4326.0,1920.0,1080.0,1920.0,1080.0,2202.907170,2202.907170,145.0,...,0,0,0,0,NaN,NaN,1995.0,male,caucasian,highSchool
1,1,1.0,4457.0,1920.0,1080.0,1920.0,1080.0,2202.907170,2202.907170,160.0,...,0,0,0,0,NaN,NaN,1995.0,male,caucasian,highSchool
2,1,1.0,4334.0,1920.0,1080.0,1920.0,1080.0,2202.907170,2202.907170,163.0,...,0,0,0,0,NaN,NaN,1995.0,male,caucasian,highSchool
3,1,1.0,4380.0,1920.0,1080.0,1920.0,1080.0,2202.907170,2202.907170,196.0,...,0,0,0,0,NaN,NaN,1995.0,male,caucasian,highSchool
4,1,1.0,3521.0,1920.0,1080.0,1920.0,1080.0,2202.907170,2202.907170,199.0,...,0,0,0,0,NaN,NaN,1995.0,male,caucasian,highSchool
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2981,97,0.0,3915.0,1280.0,800.0,1280.0,800.0,1509.436981,1509.436981,481.0,...,0,0,0,0,NaN,inf,1988.0,female,caucasian,NaN
2982,97,0.0,3964.0,1280.0,800.0,1280.0,800.0,1509.436981,1509.436981,490.0,...,0,0,0,0,NaN,inf,1988.0,female,caucasian,NaN
2983,97,0.0,3933.0,1280.0,800.0,1280.0,800.0,1509.436981,1509.436981,493.0,...,0,0,0,0,NaN,inf,1988.0,female,caucasian,NaN
2984,97,0.0,3928.0,1280.0,800.0,1280.0,800.0,1509.436981,1509.436981,502.0,...,0,0,0,0,NaN,inf,1988.0,female,caucasian,NaN


# Export data

In [75]:
if not os.path.exists('./data_jupyter'):
    os.mkdir('./data_jupyter')

data_et.to_csv("data_jupyter/data_et.csv", index=False, header=True)
data_et_fixation.to_csv("data_jupyter/data_et_fixation.csv", index=False, header=True)
data_et_choice.to_csv("data_jupyter/data_et_choice.csv", index=False, header=True)

data_trial.to_csv("data_jupyter/data_trial.csv", index=False, header=True)
data_trial_fixation.to_csv("data_jupyter/data_trial_fixation.csv", index=False, header=True)
data_trial_choice.to_csv("data_jupyter/data_trial_choice.csv", index=False, header=True)

data_subject.to_csv("data_jupyter/data_subject.csv", index=False, header=True)

MatLab input

In [76]:
if not os.path.exists('./amasino_dataPrep/data_source'):
    os.mkdir('./amasino_dataPrep/data_source')

data_et_choice['fixationCounter'] = 1
data_et_choice.loc[:, 
                       [
                           'run_id', 
                           'withinTaskIndex', 
                           'x', 
                           'y', 
                           't_task', 
                           'window_width', 
                           'window_height',
                       ]
                  ] \
   .to_csv("amasino_dataPrep/data_source/schneegansEtAl_ET.csv", index=False, header=False)

In [77]:
data_trial_choice.loc[:, 
                          [
                              'run_id', 
                              'aSS', 
                              'aLL', 
                              'tSS', 
                              'tLL', 
                              'choseLL', 
                              'trial_duration_exact', 
                              'LL_top',
                              'choseTop'
                          ]
                     ] \
    .to_csv("amasino_dataPrep/data_source/schneegansEtAl_behavior.csv", index=False, header=False)

In [78]:
data_trial_choice.loc[:, 
                       [
                           'run_id', 
                           'withinTaskIndex', 
                           'optionIndex', 
                           'attributeIndex', 
                           'payneIndex'
                       ]
                  ] \
    .fillna(0) \
    .to_csv("amasino_dataPrep/intermediateCSVs/ET_indices.csv", index=False, header=False)

In [79]:
data_subject.loc[:, ['run_id', 'choseLL']] \
    .to_csv("amasino_dataPrep/intermediateCSVs/percLeft.csv", index=False, header=False)

# Feedback

In [80]:
print('Success! Script ran through')

Success! Script ran through
